# ENGINEERING CALCULATION SHEET

**Project:** {{project_name}}  
**Project No:** {{project_number}}  
**Subject:** Lime Softening Design Calculation  
**Calc ID:** {{calc_id}}  
**Revision:** {{revision}}  
**Date:** {{date}}  

| Prepared By | Checked By | Approved By |
|------------|------------|-------------|
| {{preparer}} | {{checker}} | {{approver}} |
| {{prep_date}} | {{check_date}} | {{approve_date}} |

In [ ]:
# Papermill parameters
project_name = "Example Water Treatment Plant"
project_number = "2024-001"
calc_id = "CALC-LS-001"
revision = "0"
date = "2024-01-15"
preparer = "AI Assistant"
checker = "Senior Engineer"
approver = "Project Manager"
prep_date = "2024-01-15"
check_date = "-"
approve_date = "-"

# Calculation data
calculation_data = {}

## 1. OBJECTIVE

To determine the optimal lime (Ca(OH)₂) dosage for softening the raw water to achieve:
- Total hardness < 50 mg/L as CaCO₃
- Compliance with drinking water standards
- Minimal chemical usage and sludge production

## 2. DESIGN CRITERIA

- **Flow Rate:** {{flow_rate}} m³/h
- **Target Hardness:** < 50 mg/L as CaCO₃
- **Operating pH:** 10.0 - 10.5
- **Contact Time:** 30 minutes minimum
- **Temperature:** {{temperature}}°C

## 3. REFERENCES

1. AWWA/ASCE Water Treatment Plant Design, 5th Edition
2. MWH Water Treatment: Principles and Design, 3rd Edition  
3. PHREEQC Version 3.8.6 - USGS
4. Standard Methods for Examination of Water and Wastewater, 23rd Edition

## 4. INPUT WATER QUALITY

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import display, HTML, Markdown

# Extract input water quality
input_water = calculation_data.get('inputs', {})
analysis = input_water.get('analysis', {})

# Create formatted table
parameters = []
for param, value in analysis.items():
    # Convert to appropriate units
    if param == 'Ca':
        parameters.append({
            'Parameter': 'Calcium',
            'Value': f"{value:.2f}",
            'Unit': 'mmol/L',
            'As mg/L': f"{value * 40.08:.1f}"
        })
    elif param == 'Mg':
        parameters.append({
            'Parameter': 'Magnesium',
            'Value': f"{value:.2f}",
            'Unit': 'mmol/L',
            'As mg/L': f"{value * 24.31:.1f}"
        })
    elif param == 'Alkalinity' or param == 'C(4)':
        parameters.append({
            'Parameter': 'Alkalinity',
            'Value': f"{value:.2f}",
            'Unit': 'mmol/L',
            'As mg/L CaCO₃': f"{value * 50.04:.1f}"
        })
    elif param == 'pH':
        parameters.append({
            'Parameter': 'pH',
            'Value': f"{value:.2f}",
            'Unit': '-',
            'As mg/L': '-'
        })

input_df = pd.DataFrame(parameters)
display(HTML("<b>Table 1: Raw Water Quality Analysis</b>"))
display(input_df.style.hide(axis='index').set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('text-align', 'left'), ('padding', '8px')]}
]))

## 5. METHODOLOGY

### 5.1 Lime Softening Chemistry

The fundamental reactions for lime softening are:

**Carbonate Hardness Removal:**
$$\text{Ca}^{2+} + 2\text{HCO}_3^- + \text{Ca(OH)}_2 \rightarrow 2\text{CaCO}_3 \downarrow + 2\text{H}_2\text{O}$$

**Magnesium Removal:**
$$\text{Mg}^{2+} + \text{Ca(OH)}_2 \rightarrow \text{Mg(OH)}_2 \downarrow + \text{Ca}^{2+}$$

**Non-Carbonate Hardness (if soda ash added):**
$$\text{Ca}^{2+} + \text{Na}_2\text{CO}_3 \rightarrow \text{CaCO}_3 \downarrow + 2\text{Na}^+$$

### 5.2 Design Approach

1. Calculate carbonate and non-carbonate hardness
2. Determine theoretical lime requirement
3. Apply practical excess factor (10-20%)
4. Model using PHREEQC for equilibrium
5. Verify residual hardness meets target
6. Calculate sludge production

## 6. CALCULATIONS

### 6.1 Hardness Classification

In [ ]:
# Extract values
ca_mmol = analysis.get('Ca', 0)
mg_mmol = analysis.get('Mg', 0)
alk_mmol = analysis.get('Alkalinity', analysis.get('C(4)', 0))

# Calculate hardness in mg/L as CaCO3
ca_hardness = ca_mmol * 50.04  # Ca as CaCO3
mg_hardness = mg_mmol * 50.04 * (24.31/20.04)  # Mg as CaCO3
total_hardness = ca_hardness + mg_hardness
alkalinity_caco3 = alk_mmol * 50.04

# Determine carbonate and non-carbonate hardness
if total_hardness <= alkalinity_caco3:
    carbonate_hardness = total_hardness
    non_carbonate_hardness = 0
else:
    carbonate_hardness = alkalinity_caco3
    non_carbonate_hardness = total_hardness - alkalinity_caco3

print("=== Hardness Analysis ===")
print(f"Calcium Hardness:        {ca_hardness:.1f} mg/L as CaCO₃")
print(f"Magnesium Hardness:      {mg_hardness:.1f} mg/L as CaCO₃")
print(f"Total Hardness:          {total_hardness:.1f} mg/L as CaCO₃")
print(f"Alkalinity:              {alkalinity_caco3:.1f} mg/L as CaCO₃")
print(f"\nCarbonate Hardness:      {carbonate_hardness:.1f} mg/L as CaCO₃")
print(f"Non-Carbonate Hardness:  {non_carbonate_hardness:.1f} mg/L as CaCO₃")

### 6.2 Theoretical Lime Requirement

In [ ]:
# Theoretical lime calculations
print("=== Theoretical Lime Requirement ===")

# Lime for carbonate hardness
lime_carb = alk_mmol  # 1:1 with alkalinity
print(f"For carbonate hardness:   {lime_carb:.2f} mmol/L")

# Lime for magnesium
lime_mg = mg_mmol  # 1:1 with Mg
print(f"For magnesium removal:    {lime_mg:.2f} mmol/L")

# Lime for raising pH to 10.3
lime_pH = 0.5  # Empirical for pH adjustment
print(f"For pH adjustment:        {lime_pH:.2f} mmol/L")

# Total theoretical
lime_theoretical = lime_carb + lime_mg + lime_pH
print(f"\nTotal theoretical:        {lime_theoretical:.2f} mmol/L")

# Apply excess factor
excess_factor = 1.15  # 15% excess
lime_practical = lime_theoretical * excess_factor
print(f"With {(excess_factor-1)*100:.0f}% excess:           {lime_practical:.2f} mmol/L")
print(f"                          {lime_practical * 74.09:.1f} mg/L Ca(OH)₂")

### 6.3 PHREEQC Equilibrium Modeling

In [ ]:
# Display PHREEQC simulation results
treatment_results = calculation_data.get('treatment_results', {})
solution_summary = treatment_results.get('solution_summary', {})

print("=== PHREEQC Simulation Results ===")
print(f"Lime dose applied:        {treatment_results.get('reagent_added_mmol', 0):.2f} mmol/L")
print(f"Final pH:                 {solution_summary.get('pH', 0):.2f}")
print(f"Final ionic strength:     {solution_summary.get('ionic_strength', 0):.4f} mol/L")

# Precipitated phases
precipitates = treatment_results.get('precipitated_phases', {})
if precipitates:
    print("\nPrecipitated Minerals:")
    total_precip = 0
    for mineral, amount in precipitates.items():
        mass_g_L = amount * {'Calcite': 100.09, 'Brucite': 58.32}.get(mineral, 100)
        print(f"  {mineral:15s} {amount:8.4f} mol/L   {mass_g_L:6.2f} g/L")
        total_precip += mass_g_L
    print(f"\nTotal precipitate:        {total_precip:.2f} g/L")

### 6.4 Final Water Quality

In [ ]:
# Extract final water quality
final_analysis = solution_summary.get('analysis', {})

# Calculate final hardness
final_ca = final_analysis.get('Ca', 0) * 50.04
final_mg = final_analysis.get('Mg', 0) * 50.04 * (24.31/20.04)
final_hardness = final_ca + final_mg

# Create comparison table
comparison_data = [
    {'Parameter': 'pH', 'Initial': input_water.get('pH', 0), 'Final': solution_summary.get('pH', 0), 'Target': '10.0-10.5', 'Unit': '-'},
    {'Parameter': 'Total Hardness', 'Initial': f"{total_hardness:.0f}", 'Final': f"{final_hardness:.0f}", 'Target': '<50', 'Unit': 'mg/L CaCO₃'},
    {'Parameter': 'Calcium', 'Initial': f"{ca_mmol*40.08:.0f}", 'Final': f"{final_analysis.get('Ca', 0)*40.08:.0f}", 'Target': '-', 'Unit': 'mg/L'},
    {'Parameter': 'Magnesium', 'Initial': f"{mg_mmol*24.31:.0f}", 'Final': f"{final_analysis.get('Mg', 0)*24.31:.0f}", 'Target': '-', 'Unit': 'mg/L'},
]

comp_df = pd.DataFrame(comparison_data)
display(HTML("<b>Table 2: Water Quality Comparison</b>"))
display(comp_df.style.hide(axis='index').set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('text-align', 'center'), ('padding', '8px')]}
]))

# Calculate removal efficiency
if total_hardness > 0:
    hardness_removal = (total_hardness - final_hardness) / total_hardness * 100
    print(f"\nHardness Removal Efficiency: {hardness_removal:.1f}%")
else:
    print("\nHardness Removal Efficiency: N/A (no initial hardness)")

## 7. DESIGN SUMMARY

In [ ]:
# Design calculations
flow_rate = calculation_data.get('design_params', {}).get('flow_rate', 100)  # m³/h
lime_dose_kg_h = lime_practical * 74.09 * flow_rate / 1000
sludge_production = treatment_results.get('total_precipitate_g_L', 0) * flow_rate / 1000

design_box = f"""
<div style="border: 2px solid black; padding: 15px; margin: 20px 0; background-color: #f5f5f5;">
<h3 style="margin-top: 0;">DESIGN PARAMETERS</h3>
<table style="width: 100%; border-collapse: collapse;">
<tr><td style="padding: 5px;"><b>Design Flow Rate:</b></td><td style="text-align: right;">{flow_rate} m³/h</td></tr>
<tr><td style="padding: 5px;"><b>Lime Dose:</b></td><td style="text-align: right;">{lime_practical * 74.09:.0f} mg/L</td></tr>
<tr><td style="padding: 5px;"><b>Lime Consumption:</b></td><td style="text-align: right;">{lime_dose_kg_h:.1f} kg/h</td></tr>
<tr><td style="padding: 5px;"><b>Daily Lime Usage:</b></td><td style="text-align: right;">{lime_dose_kg_h * 24:.0f} kg/day</td></tr>
<tr><td style="padding: 5px;"><b>Sludge Production:</b></td><td style="text-align: right;">{sludge_production:.1f} kg/h (dry basis)</td></tr>
<tr><td style="padding: 5px;"><b>Contact Time Required:</b></td><td style="text-align: right;">30 minutes</td></tr>
<tr><td style="padding: 5px;"><b>Reactor Volume:</b></td><td style="text-align: right;">{flow_rate * 0.5:.0f} m³</td></tr>
</table>
</div>
"""
display(HTML(design_box))

## 8. CONCLUSIONS

1. The recommended lime dose of **{{lime_dose}} mg/L Ca(OH)₂** achieves the target hardness reduction
2. Final hardness of **{{final_hardness}} mg/L as CaCO₃** meets the design criteria of <50 mg/L
3. Hardness removal efficiency of **{{removal_efficiency}}%** is achieved
4. Estimated sludge production is **{{sludge_rate}} kg/h** (dry basis)

## 9. RECOMMENDATIONS

1. **Chemical Feed System:**
   - Install lime slurry system with capacity for {{lime_capacity}} kg/h
   - Provide 20% excess capacity for peak flows
   - Include automatic pH control for optimization

2. **Reactor Design:**
   - Minimum contact time: 30 minutes
   - Provide mechanical mixing at 50-75 W/m³
   - Include provisions for sludge recirculation

3. **Sludge Handling:**
   - Design for {{sludge_daily}} kg/day dry solids
   - Provide thickening to 3-5% solids
   - Consider lime sludge recovery if economical

4. **Process Control:**
   - Monitor influent and effluent hardness
   - Maintain pH between 10.0-10.5
   - Optimize lime dose based on actual removal

## APPENDIX A: SATURATION INDICES

In [ ]:
# Display saturation indices
initial_si = calculation_data.get('speciation_results', {}).get('saturation_indices', {})
final_si = treatment_results.get('saturation_indices', {})

si_data = []
for mineral in ['Calcite', 'Aragonite', 'Brucite', 'Gypsum']:
    si_data.append({
        'Mineral': mineral,
        'Initial SI': f"{initial_si.get(mineral, -999):.2f}",
        'Final SI': f"{final_si.get(mineral, -999):.2f}",
        'Status': 'Precipitating' if final_si.get(mineral, -999) > 0 else 'Undersaturated'
    })

si_df = pd.DataFrame(si_data)
display(HTML("<b>Table A1: Mineral Saturation Indices</b>"))
display(si_df.style.hide(axis='index'))

## APPENDIX B: PHREEQC INPUT FILE

In [ ]:
# Display PHREEQC input if available
phreeqc_input = calculation_data.get('phreeqc_input', '')
if phreeqc_input:
    print("PHREEQC Input File:")
    print("=" * 60)
    print(phreeqc_input)
else:
    print("PHREEQC input file not available in calculation data.")

---
*End of Calculation Sheet*